# Benchmark: Koalas (PySpark) and Dask - Data Preparation
The benchmark was performed against the 2009 - 2013 Yellow Taxi Trip Records (157 GB) from NYC Taxi and Limousine Commission (TLC) Trip Record Data.

The CSV files were downloaded into Databricks File System (DBFS), and then were converted into Parquet files via Koalas for better efficiency.

Download url: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page.

Data dictionary: https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf.

The scenario used in this benchmark was inspired by https://github.com/xdssio/big_data_benchmarks.

# Download CSV files to DBFS

In [1]:
%%sh
sudo mkdir -p ../../datasets/taxi_csv/

Couldn't find program: 'sh'


In [2]:
%%sh
ls ../../datasets/taxi_csv

Couldn't find program: 'sh'


The old download version didn't work, changed it to a new one using "download_taxi_datasets.py"

In [3]:
# url_loc = {} # Map download url to the file location in DBFS

# for year in range(2009, 2014):
#   for m in range(1, 13):
#     month = "{:02d}".format(m)
#     fname = 'yellow_tripdata_%s-%s.csv' % (year, month)
#     url = 'https://s3.amazonaws.com/nyc-tlc/trip+data/%s' % fname
#     loc = f'{DATASETS_DIR}/taxi_csv/%s' % fname
#     url_loc[url] = loc

In [4]:
# import urllib.request

# for url, loc in url_loc.items():
#   urllib.request.urlretrieve(url, loc)

Using our version to download data

<span style="font-size:20pt"><b>NOTE:</b> </span> the dataset changed so we had to change the years of this analysis... Initialy this was between 2009 and 2014, now it's from 2011 until 2015... (The columns changed from 2011 forward)

In [5]:
import datetime as dt
import os

# Add "code" folder to path so we can import the "download_taxi_files" script
import sys
sys.path.insert(1, os.path.abspath('..'))
from download_taxi_datasets import download_taxi_files    

DATASETS_DIR = '../../datasets'
years = range(2011, 2013)
date_range = [dt.datetime(year, month, 1) for year in years for month in range(1, 13)]
files = download_taxi_files(date_range, f'{DATASETS_DIR}/taxi_csv')

100%|██████████| 24/24 [00:02<00:00,  8.83it/s]


In [6]:
%%sh
ls ../../datasets/taxi_csv

Couldn't find program: 'sh'


In [7]:
# Changed bellow section to work in a normal Linux machine
# total_bytes = 0
# for fileInfo in dbutils.fs.ls(f'{OUTPUT_FOLDER}/taxi_csv'):
#   total_bytes += fileInfo.size
import os
total_bytes = 0
for filename in os.listdir(f'{DATASETS_DIR}/taxi_csv'):
    file_path = os.path.join(f'{DATASETS_DIR}/taxi_csv', filename)
    if os.path.isfile(file_path): total_bytes += os.path.getsize(file_path)
print('%s GBs data in total' % (total_bytes * 1e-9))

4.392001261 GBs data in total


# Convert to Parquet files
Convert downloaded CSV files into Parquet files via Koalas for better efficiency.

In [8]:
# import databricks.koalas as ks
 
# ks.set_option('compute.default_index_type', 'distributed-sequence')

# Instead of using koalas to rename the columns, we're going to use dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

# cluster = LocalCluster(n_workers=4, threads_per_worker=3, memory_limit='30GiB')
cluster = LocalCluster(n_workers=1, threads_per_worker=2, memory_limit='20GiB')
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:63370 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 2 Memory: 21.47 GB


## Changing columns names

In order for the datasets to match the format of the ones in the article, we had to change some of the column names

In [9]:
renaming_cols = {
  'passenger_count': 'Passenger_Count',
  'fare_amount': 'Fare_Amt',
  'tip_amount': 'Tip_Amt',
  # The 2 renamings bellow, don't show the exact LAT/LON, 
  # but we only want to benchmark, so they're not entirely relevant...
  # We'll later create the respective Lat coordinates by copying the values of this lon ones
  'PULocationID': 'Start_Lon',
  'DOLocationID': 'End_Lon',

  'tolls_amount': 'Tolls_Amt',
  'total_amount': 'Total_Amt',
}

In [10]:
dtype_dict = {
  'Passenger_Count': 'int64', 
  'Start_Lon': 'float64', 
  'Start_Lat': 'float64',
  'End_Lon': 'float64', 
  'End_Lat': 'float64', 
  'Fare_Amt': 'float64', 
  'Tip_Amt': 'float64', 
  'Tolls_Amt': 'float64',
  'Total_Amt': 'float64'
}
# Convert Article dtype_dict to new one
dtype_dict = {col: dtype_dict[renaming_cols[col]] for col in renaming_cols.keys()}
ks_df = dd.read_parquet(f'{DATASETS_DIR}/taxi_csv', dtype=dtype_dict)

# Drop all other columns not relevant (to us)
ks_df = ks_df.drop(columns=ks_df.columns.difference(dtype_dict.keys()))

In [11]:
# Renaming columns to match old format
ks_df = ks_df.rename(columns=renaming_cols)

# Create the Lat/Lon columns
ks_df['Start_Lat'] = ks_df['Start_Lon'].copy()
ks_df['End_Lat'] = ks_df['End_Lon'].copy()

ks_df.columns = ks_df.columns.str.lower()

In [12]:
%%sh
rm -fr ../../datasets/ks_taxi_parquet

Couldn't find program: 'sh'


In [13]:
ks_df.to_parquet(f'{DATASETS_DIR}/ks_taxi_parquet')

In [14]:
# total_bytes = 0
# for file_info in dbutils.fs.ls('FileStore/ks_taxi_parquet'):
#   total_bytes += file_info.size
# print('%s GBs data in total' % (total_bytes * 1e-9))

total_bytes = 0
for filename in os.listdir(f'{DATASETS_DIR}/ks_taxi_parquet'):
    file_path = os.path.join(f'{DATASETS_DIR}/ks_taxi_parquet', filename)
    if os.path.isfile(file_path): total_bytes += os.path.getsize(file_path)
print('%s GBs data in total' % (total_bytes * 1e-9))

6.179561619 GBs data in total


# Note: Filtering Size
(Size of filtered data / Size of total data) in the benchmark

In [ ]:
import databricks.koalas as ks
koalas_data = ks.read_parquet(f'{DATASETS_DIR}/ks_taxi_parquet')

In [14]:
expr_filter = (koalas_data.tip_amt >= 1) & (koalas_data.tip_amt <= 5)
 
print(f'In the benchmark, filtered data is {len(koalas_data[expr_filter]) / len(koalas_data) * 100}% of total data')

In the benchmark, filtered data is 45.102692369637836% of total data
